In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import joblib

In [ ]:
df_train_validation = pd.read_csv("final_train_val.csv", low_memory=False, index_col="id")
df_test = pd.read_csv("final_test.csv", low_memory=False, index_col="id")

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
df_train_validation.columns

Index(['MONTH', 'DAY', 'DAY_OF_WEEK', 'FLIGHT_NUMBER', 'SCHEDULED_DEPARTURE',
       'TAXI_OUT', 'SCHEDULED_ARRIVAL', 'ARRIVAL_DELAY', 'LATITUDE_origin',
       'LONGITUDE_origin', 'LATITUDE_arrival', 'LONGITUDE_arrival',
       'AIRLINE_AA', 'AIRLINE_AS', 'AIRLINE_B6', 'AIRLINE_DL', 'AIRLINE_EV',
       'AIRLINE_F9', 'AIRLINE_HA', 'AIRLINE_MQ', 'AIRLINE_NK', 'AIRLINE_OO',
       'AIRLINE_UA', 'AIRLINE_US', 'AIRLINE_VX', 'AIRLINE_WN',
       'scaled_DEPARTURE_TIME', 'scaled_WHEELS_OFF', 'scaled_SCHEDULED_TIME',
       'scaled_DISTANCE'],
      dtype='object')

In [ ]:
df_test.head()

,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,TAXI_OUT,SCHEDULED_ARRIVAL,LATITUDE_origin,LONGITUDE_origin,LATITUDE_arrival,LONGITUDE_arrival,AIRLINE_AA,AIRLINE_AS,AIRLINE_B6,AIRLINE_DL,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN,scaled_DEPARTURE_TIME,scaled_WHEELS_OFF,scaled_SCHEDULED_TIME,scaled_DISTANCE
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,7,1,3,298,3,11.0,500,36.08036,-115.15233,29.98047,-95.33972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.578947,-1.583333,0.606742,0.832849
1,7,1,3,1965,5,21.0,808,37.61900,-122.37484,35.21401,-80.94313,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.308446,-1.621324,2.022472,2.393895
2,7,1,3,2020,5,36.0,548,33.94254,-118.40807,35.04242,-89.97667,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.308446,-1.602941,1.123596,1.409884
3,7,1,3,612,5,14.0,503,36.08036,-115.15233,44.88055,-93.21692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.564259,-1.515931,0.617978,0.944767
4,7,1,3,1044,8,20.0,912,61.17432,-149.99619,41.97960,-87.90446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.608323,-1.601716,2.707865,3.193314


In [ ]:
pd.read_csv("flights_test.csv").head()

,id,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL
0,0,2015,7,1,3,NK,298,N624NK,LAS,IAH,3,41.0,11.0,52.0,177.0,1222,500
1,1,2015,7,1,3,AA,1965,N507AY,SFO,CLT,5,2400.0,21.0,21.0,303.0,2296,808
2,2,2015,7,1,3,DL,2020,N3759,LAX,MEM,5,2400.0,36.0,36.0,223.0,1619,548
3,3,2015,7,1,3,NK,612,N629NK,LAS,MSP,5,53.0,14.0,107.0,178.0,1299,503
4,4,2015,7,1,3,UA,1044,N39418,ANC,ORD,8,17.0,20.0,37.0,364.0,2846,912


In [ ]:
def missingValuesInfo(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = round(df.isnull().sum().sort_values(ascending = False)/len(df)*100, 2)
    temp = pd.concat([total, percent], axis = 1,keys= ['Total', 'Percent'])
    return temp.loc[(temp['Total'] > 0)]

missingValuesInfo(df_train_validation)

,Total,Percent


In [ ]:
# Split: for XGBoost
df_train, df_validation = train_test_split(df_train_validation, test_size=0.20, random_state = 42)
X_train, y_train = df_train.drop("ARRIVAL_DELAY", axis=1), df_train["ARRIVAL_DELAY"]
X_val, y_val = df_validation.drop("ARRIVAL_DELAY", axis=1), df_validation["ARRIVAL_DELAY"]
X_test = df_test

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
# Hyperparameter tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe


space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }

def objective(space):
    reg = xgb.XGBRegressor(n_estimators =space['n_estimators'], 
                           max_depth = int(space['max_depth']),
                           gamma = space['gamma'], 
                           reg_alpha = int(space['reg_alpha']),
                           min_child_weight=int(space['min_child_weight']),
                           colsample_bytree=int(space['colsample_bytree']))

    eval_set  = [(X_train, y_train), (X_val, y_val)]

    reg.fit(X_train, y_train, eval_set=eval_set, eval_metric = 'rmse',
						early_stopping_rounds=10,verbose=False)
    val_pred = reg.predict(X_val)
    mse = mean_squared_error(y_val, val_pred)
    return{'loss':mse, 'status': STATUS_OK }

trials = Trials()
best_hyperparams = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=5,
            trials=trials)

print(best_hyperparams)

print("The best hyperparameters are : ","\n")
print(best_hyperparams)

In [ ]:
best_hyperparams = {'colsample_bytree': 0.5792280519850819, 'gamma': 2.066360196498205, 'max_depth': 17, 'min_child_weight': 9, 'reg_alpha': 169, 'reg_lambda': 0.8100274580712421}

xg_reg = xgb.XGBRegressor(**best_hyperparams)
xg_reg.fit(X_train,y_train)
val_pred = xg_reg.predict(X_val)
mse = mean_squared_error(y_val, val_pred)
print(mse)

[16:48:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
10.00209335087714


In [ ]:
joblib.dump(xg_reg, "xgreg.pkl")

['xgreg.pkl']

In [ ]:
eval_pred = xg_reg.predict(X_test)

In [ ]:
# GradientBoostingRegressor
# build our model scoring function
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)
def cv_mse(model, X, y):
    mse = - cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kfolds)
    return (mse)

X_train_cv, y_train_cv = df_train_validation.drop("ARRIVAL_DELAY", axis=1), df_train_validation["ARRIVAL_DELAY"]
gbr = GradientBoostingRegressor()
score = cv_mse(gbr, X_train_cv, y_train_cv)
print(score)

In [ ]:
gbr.fit(X_train_cv, y_train_cv)
joblib.dump(gbr, "gradient_boosting_regressor.pkl")

In [ ]:
pd.DataFrame(eval_pred, columns=['ARRIVAL_DELAY']).to_csv("flight_result.csv", index_label='id')